# Implementing a port to optuna optimizers

In this example, we will show how you can implement your own methods from 3rd party libraries into the f3dasm abstractions. We demonstrate this with the [`optuna`](https://optuna.org/)optimizers.

In [1]:
from f3dasm import Block, DataGenerator, ExperimentData, ExperimentSample
from f3dasm.design import Domain
from f3dasm._src.design.parameter import ContinuousParameter, DiscreteParameter, CategoricalParameter, ConstantParameter, ArrayParameter

Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It contains various optimization algorithms and is used mostly for hyperparameter optimization tasks. One of their implemented algorithms i the `optuna.samplers.TPESampler`, which is a tree-structured Parzen estimator sampler. It is a sequential model-based optimization algorithm that builds a probabilistic model of the objective function and uses it to sample new points in the parameter space.

If we want to use this sampler as an optimization algorithm in the f3dasm framework, we have to make sure that it implements the `Block` interface. Since `optuna` has it's own interfaces, we have to translate the `optuna` interfaces to the `f3dasm` interfaces. After that, we can use the `optuna` sampler as an optimization algorithm in the f3dasm framework.

This tutorial is a step-by-step guide on how to implement the `optuna` sampler as a `Block` in the f3dasm framework, and can be used as a template for implementing other 3rd party libraries as `Block`s in the f3dasm framework.

In [2]:
import optuna

## Domain to `optuna.distribution` mapping

`Optuna` uses its own distribution classes to represent the parameter space. The `optuna.distributions` module provides various distributions that can be used to define the search space for the optimization problem. In `f3dasm`, we use the `Domain` class to represent the parameter space. We need to map the `Domain` to the `optuna.distributions`:

In [3]:
def domain_to_optuna_distributions(domain: Domain) -> dict:
    """
    Convert a domain object to Optuna distributions.

    Parameters
    ----------
    domain : Domain
        The domain describing the input space.

    Returns
    -------
    dict
        Dictionary mapping parameter names to Optuna distributions.
    """
    optuna_distributions = {}
    for name, parameter in domain.input_space.items():
        if isinstance(parameter, ContinuousParameter):
            optuna_distributions[name] = \
                optuna.distributions.FloatDistribution(
                low=parameter.lower_bound,
                high=parameter.upper_bound, log=parameter.log)
        elif isinstance(parameter, DiscreteParameter):
            optuna_distributions[name] = \
                optuna.distributions.IntDistribution(
                low=parameter.lower_bound,
                high=parameter.upper_bound, step=parameter.step)
        elif isinstance(parameter, CategoricalParameter):
            optuna_distributions[name] = \
                optuna.distributions.CategoricalDistribution(
                parameter.categories)
        elif isinstance(parameter, ConstantParameter):
            optuna_distributions[name] = \
                optuna.distributions.CategoricalDistribution(
                choices=[parameter.value])
        elif isinstance(parameter, ArrayParameter):
            raise ValueError(
                "ArrayParameter is not supported in Optuna trials. "
                "Please use a different parameter types in your Domain."
            )
        else:
            raise TypeError(
                f"Unsupported parameter type: {type(parameter)} for {name}")
    return optuna_distributions

We go over each of the parameters in the `Domain` and map them to the corresponding `optuna.distributions`

## ExperimentSample to `optuna.Trial`

Optuna uses the `optuna.Trial` class to represent a single trial in the optimization process. A trial is a single evaluation of the objective function with a specific set of hyperparameters. In `f3dasm`, we use the `ExperimentSample` class to represent a single experiment sample. We need to map the `ExperimentSample` to the `optuna.Trial`:

In [4]:

def _suggest_experimentsample(trial: optuna.Trial,
                              domain: Domain) -> ExperimentSample:
    """
    Suggest a new experiment sample using Optuna trial and domain.

    Parameters
    ----------
    trial : optuna.Trial
        The Optuna trial object for suggesting parameters.
    domain : Domain
        The domain describing the input space.

    Returns
    -------
    ExperimentSample
        The suggested experiment sample.
    """
    optuna_dict = {}
    for name, parameter in domain.input_space.items():
        if isinstance(parameter, ContinuousParameter):
            optuna_dict[name] = trial.suggest_float(
                name=name,
                low=parameter.lower_bound,
                high=parameter.upper_bound, log=parameter.log)
        elif isinstance(parameter, DiscreteParameter):
            optuna_dict[name] = trial.suggest_int(
                name=name,
                low=parameter.lower_bound,
                high=parameter.upper_bound, step=parameter.step)
        elif isinstance(parameter, CategoricalParameter):
            optuna_dict[name] = trial.suggest_categorical(
                name=name,
                choices=parameter.categories)
        elif isinstance(parameter, ConstantParameter):
            optuna_dict[name] = trial.suggest_categorical(
                name=name, choices=[parameter.value])
        elif isinstance(parameter, ArrayParameter):
            raise ValueError(
                "ArrayParameter is not supported in Optuna trials. "
                "Please use a different parameter types in your Domain."
            )
        else:
            raise TypeError(
                f"Unsupported parameter type: {type(parameter)} "
                f"for {name}")
    return ExperimentSample(input_data=optuna_dict, domain=domain)

We go over each of the parameters in the `ExperimentSample` and map them to the corresponding `optuna.Trial` methods. The `optuna.Trial` class provides methods to suggest hyperparameters, which we can use to set the values of the `ExperimentSample`.

## Putting it all together

We implement the `optuna` sampler as a `Block` in the f3dasm framework. The `optuna` sampler will use the `Domain` and `ExperimentSample` classes to represent the parameter space and the experiment samples, respectively. We will implement the `Block` interface and use the `optuna` sampler to sample new points in the parameter space. The samples will be evaluated by a `DataGenerator` that will return the objective function value for the given sample. We can `call` the `optuna` sampler with the initial samples as an `ExperimentData` object and the number of update steps (`n_iterations`) to perform. The `optuna` sampler will return the updated `ExperimentData` object with the new samples.

In [5]:
class OptunaOptimizer(Block):
    """
    Optuna-based optimizer block for experiment design and optimization.

    This class wraps Optuna's optimization logic and integrates it with the
    experiment data and domain definitions from f3dasm.
    """

    def __init__(self, optuna_sampler: optuna.distributions.BaseDistribution):
        """
        Initialize the OptunaOptimizer.

        Parameters
        ----------
        optuna_sampler : optuna.distributions.BaseDistribution
            The Optuna sampler to use for the optimization process.
        """
        self.optuna_sampler = optuna_sampler

    def arm(self, data: ExperimentData, data_generator: DataGenerator,
            output_name: str):
        """
        Prepare the optimizer with experiment data, data generator,
        and output name.

        Parameters
        ----------
        data : ExperimentData
            The experiment data containing previous trials.
        data_generator : DataGenerator
            The data generator used to evaluate new samples.
        output_name : str
            The name of the output variable to optimize.
        """
        self.data_generator = data_generator
        self.output_name = output_name
        self.distributions = domain_to_optuna_distributions(
            data.domain)

        # Set algorithm
        self.study = optuna.create_study(
            sampler=self.optuna_sampler
        )

        # Add existing trials to the study
        for _, es in data:
            self.study.add_trial(
                optuna.trial.create_trial(
                    params=es.input_data,
                    distributions=self.distributions,
                    value=es.output_data[self.output_name],
                )
            )

    def _step(self, data: ExperimentData, **kwargs) -> ExperimentData:
        """
        Perform a single optimization step.

        This method suggests a new experiment sample, evaluates it, and adds
        the result to the Optuna study.

        Parameters
        ----------
        data : ExperimentData
            The current experiment data.
        **kwargs
            Additional arguments passed to the data generator.

        Returns
        -------
        ExperimentData
            The updated experiment data including the new sample.
        """
        trial = self.study.ask()
        new_es = _suggest_experimentsample(
            trial=trial, domain=data.domain)

        new_experiment_data = ExperimentData.from_data(
            data={0: new_es},
            domain=data.domain,
            project_dir=data.project_dir)

        # Evaluate the sample with the data generator
        self.data_generator.arm(data=new_experiment_data)
        new_experiment_data = self.data_generator.call(
            data=new_experiment_data, **kwargs)

        # TODO: extract last es
        new_es = new_experiment_data.get_experiment_sample(
            new_experiment_data.index[-1])

        self.study.add_trial(
            optuna.trial.create_trial(
                params=new_es.input_data,
                distributions=self.distributions,
                value=new_es.output_data[self.output_name],
            )
        )

        return new_experiment_data

    def call(self, data: ExperimentData,
             n_iterations: int, **kwargs) -> ExperimentData:
        """
        Run the optimization for a specified number of iterations.

        Parameters
        ----------
        data : ExperimentData
            The initial experiment data.
        n_iterations : int
            Number of optimization steps to perform.
        **kwargs
            Additional arguments passed to each optimization step.

        Returns
        -------
        ExperimentData
            The experiment data after optimization.
        """
        for _ in range(n_iterations):
            data += self._step(data, **kwargs)
        return data


## Example implementation

In [6]:
from f3dasm import datagenerator
from f3dasm import create_sampler


# Create a simple data generator function
# that computes y = x^2 + 1 for a given x.
# This will be used to evaluate the samples generated by the optimizer.
@datagenerator(output_names='y')
def f(x):
    return x**2 + 1


# Create teh TPESampler from Optuna
sampler = optuna.samplers.TPESampler(seed=42)

# Create your custom OptunaOptimizer and give it the optuna sampler.
optimizer = OptunaOptimizer(
    optuna_sampler=sampler,
)

# Initialize the domain and experiment data
domain = Domain()
domain.add_float(name='x', low=-10, high=10)
domain.add_output('y')
data = ExperimentData(domain=domain)

# Sample random initial points
sampler = create_sampler('random')
data = sampler.call(data, n_samples=1)

# Evaluate the initial points using the data generator
f.arm(data)
data = f.call(data)

# Arm the optimizer with the experiment data and data generator
optimizer.arm(data, data_generator=f, output_name='y')

# Run the optimization for a specified number of iterations
data = optimizer.call(data, n_iterations=10)

[I 2025-08-15 14:54:21,407] A new study created in memory with name: no-name-e62b33fc-afcc-4869-b899-580770509471


In [7]:
data

,jobs,input,output
,,x,y
0,finished,-6.788336,47.081505
1,finished,-2.509198,7.296073
2,finished,9.014286,82.257354
3,finished,4.639879,22.528476
4,finished,1.973170,4.893399
5,finished,-6.879627,48.329270
6,finished,-6.880110,48.335908
7,finished,-8.838328,79.116038
8,finished,7.323523,54.633988
